In this turotial we will explore how to set up arbitrary lattice configurations to house qbits and assign various interactions between qbits to simulate qbit operations. Our results on energy spectrums could be benchmarks of initial qbit setups for quantum computing theories/experiments. 

## Mixed 4-site Qbits

### Introduction

We first use the 4-site mixed graph in the lattice configuration file: `lattices.xml`
```
<GRAPH name="4-site mixed" vertices="4"> 
  <VERTEX id="1" type="0"/>
  <VERTEX id="2" type="1"/>
  <VERTEX id="3" type="0"/>
  <VERTEX id="4" type="1"/>
  <EDGE type="0" source="1" target="2"/>
  <EDGE type="0" source="2" target="3"/>
  <EDGE type="0" source="3" target="4"/>
  <EDGE type="0" source="4" target="1"/>
  <EDGE type="1" source="1" target="3"/>
  <EDGE type="1" source="2" target="4"/>
</GRAPH> 
```
In this lattice configuration there are two types of vertices, labeled as "0" for sites 1 and 3 and "1" for sites 2 and 4. For each qbit site there is a transverse magnetic field with strength Gamma. There are also two types of bonds, labeled as "0" for bonds between sites (1,2), (2,3), (3,4), and (4,1), and "1" for bonds between sites (1,3) and (2,4). For bond type "0", we will assign an interaction J1 for bond type "0" and J2 for bond type "1". All these is done in the model configuration file: `models.xml`
```
<HAMILTONIAN name="qbit operation">
  <PARAMETER name="J1" default="1"/>
  <PARAMETER name="J2" default="0.5"/>
  <BASIS ref="spin"/>
  <SITETERM site="i">
    -Gamma*Sx(i)
  </SITETERM>
  <BONDTERM source="1" target="2">
    J1*Sz(1)*Sz(2)
  </BONDTERM>
  <BONDTERM source="2" target="3">
    J1*Sz(2)*Sz(3)
  </BONDTERM>
  <BONDTERM source="3" target="4">
    J1*Sz(3)*Sz(4)
  </BONDTERM>
  <BONDTERM source="4" target="1">
    J1*Sz(4)*Sz(1)
  </BONDTERM>
  <BONDTERM source="1" target="3">
    J2*Sz(1)*Sz(3)
  </BONDTERM>
  <BONDTERM source="2" target="4">
    J2*Sz(2)*Sz(4)
  </BONDTERM>
</HAMILTONIAN>
```
With the above setups, the Hamiltonian for the 4-site qbits is given by
$$
H=J_{1} \sum_{type 0} S^i_z S^j_z + J_{2} \sum_{type 1} S^i_z S^j_z - \Gamma \sum_i S^i_x.
$$

### Simulation

We first import some modules:

In [1]:
import pyalps
import numpy as np
import matplotlib.pyplot as plt

We then set up parameters for the system and loop over the second coupling constants J2. 

In [2]:
parms = []
# Loop over second coupling constant
for J2 in [0.0, 0.5, 1.0, 1.5]:
    parms.append({
        'GRAPH'      : "4-site mixed",
        'MODEL'      : "qbit operation",
        'local_S'    : 0.5,
        'Gamma'      : 0.5,
        'J2'         : J2,
        'NUMBER_EIGENVALUES' : 5
    })

Now we set up the input files and run the simulations.

In [3]:
prefix = 'qbitenergy'
input_file = pyalps.writeInputFiles(prefix,parms)
res = pyalps.runApplication('sparsediag', input_file)
data = pyalps.loadEigenstateMeasurements(pyalps.getResultFiles(prefix=prefix))

sparsediag qbitenergy.in.xml
ALPS sparse diagonalization application version 1.1
  available from http://alps.comp-phys.org/
  copyright (c) 2003-2007 by Matthias Troyer <troyer@comp-phys.org>
 for details see the publication:
  A.F. Albuquerque et al., J. of Magn. and Magn. Materials 310, 1187 (2007).

using the ALPS parallelizing scheduler
  copyright (c) 1994-2006 by Matthias Troyer <troyer@comp-phys.org>.
  see Lecture Notes in Computer Science, Vol. 1505, p. 191 (1998).

based on the ALPS libraries version 2.3.0
  available from http://alps.comp-phys.org/
  copyright (c) 1994-2016 by the ALPS collaboration.
  Consult the web page for license details.
  For details see the publication: 
  B. Bauer et al., J. Stat. Mech. (2011) P05001.

parsing task files ... 
Starting task 1.


Starting Lanczos 
Finished Lanczos


Checking if it is finished: not yet, next check in 60 seconds ( 0% done).
Halted Simulation 1
This task took 1 seconds.
Checkpointing Simulation 1
Checkpointing Simulation 2
Checkpointing Simulation 3
Checkpointing Simulation 4
Starting task 2.


0 sectors
Starting Lanczos 
Finished Lanczos


Checking if it is finished: not yet, next check in 60 seconds ( 0% done).
Halted Simulation 2
This task took 1 seconds.
Checkpointing Simulation 1
Checkpointing Simulation 2
Checkpointing Simulation 3
Checkpointing Simulation 4
Starting task 3.


0 sectors
Starting Lanczos 
Finished Lanczos


Checking if it is finished: not yet, next check in 60 seconds ( 0% done).
Halted Simulation 3
This task took 1 seconds.
Checkpointing Simulation 1
Checkpointing Simulation 2
Checkpointing Simulation 3
Checkpointing Simulation 4
Starting task 4.


0 sectors
Starting Lanczos 
Finished Lanczos


Checking if it is finished: not yet, next check in 60 seconds ( 0% done).
Halted Simulation 4
This task took 1 seconds.
Checkpointing Simulation 1
Checkpointing Simulation 2
Checkpointing Simulation 3
Checkpointing Simulation 4
Finished with everything.


We then iterate through parameter J2 and plot 5 lowest energy levels for each J2.

In [5]:
for Lsets in data:
    J2 = pyalps.flatten(Lsets)[0].props['J2']
    # Make a list of energy values for each J2
    x = []
    allE = []
    for q in pyalps.flatten(Lsets):
        x += list(q.x)
        allE += list(q.y)
    allE = np.sort(allE)

    # Set the scatter plot label
    lbl="J2=" + str(J2)
    plt.scatter(x,allE, label=lbl)

    plt.legend()
    
plt.xlabel("level")
plt.ylabel("E")
plt.title("4-site Mixed Graph")
plt.show()
